# Function Decorators and Closures

Taken from *Fluent Python* by Luciano Ramalho, published by O'Reilly Media, Inc.

## Decorators 101

A **decorator** is a callable that takes another **function as argument** (the **decorated** function). The decorator may perform some processing with the decorated function, and returns it or replaces it with another function or callable object.

In other words, assuming an existing decorator named `decorate`, this code:

```python
@decorate
def target():
  print('running target()')
```

Has the same effect as writing this:

```python
def target():
  print('running target()')
    
target = decorate(target)
```

The end result is the same: at the end of either of these snippets, the `target` name does not necessarily refer to the original `target` function, but to whatever function is returned by `decorate(target)`.

*Example 1: A decorator usually replaces a function with a different one*

In [7]:
def deco(func):
  def inner():
    print('running inner()')
  return inner  # deco returns its inner function

@deco
def target():  # target is decorated by deco
  print('running target()')

target    # See what it points to
target()  # It actually runs inner

running inner()


## When does Python execute decorators?

Decorated function is executed right after it is defined, this is usually at *import time*, i.e., when a module is loaded by Python.

In the example 2 below, the decorator `register()` run (twice) before any other function in the module.  However, the decorated functions `f1()` and `f2()` only  run whey they are explicitly invoked.  This is the difference between what Pythonistas call `import time` and `runtime`.

*Example 2: The registration.py module*

In [9]:
registry = []  # hold references to functions decorated by @register

def register(func):  # takes a function as argument
  print('running register(%s)' % func)  # show what function is being decorated
  registry.append(func)  # append func into registry
  return func  # Must return a function for the decorated function

@register  # decorate the next function f1()
def f1():
  print('running f1()')

@register  # decorate the next function f2()
def f2():
  print('running f2()')

def f3():  # function f3() is not decorated
  print('running f3()')

def main():
  print('running main()')
  print('register ->', registry)
  f1()
  f2()
  f3()

if __name__ == '__main__':
  main()

running register(<function f1 at 0x000001BFD82C2AE8>)
running register(<function f2 at 0x000001BFD83711E0>)
running main()
register -> [<function f1 at 0x000001BFD82C2AE8>, <function f2 at 0x000001BFD83711E0>]
running f1()
running f2()
running f3()


## How decorators are commonly employed in read code

*  A real decorator is usually defined in one module and is applied to functions in other modules.

*  Most decorators may define an inner function and return it.

*Example 3: clockdeco.py*

In [13]:
# %load clockdeco.py
import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    name = func.__name__
    arg_lst = []
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r ' %(elapsed, name, arg_str, result))
    return result
  return clocked

*Example 4: factorial.py*

In [19]:
# %load factorial.py
from clockdeco import clock

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
  factorial(30)

[0.00000000s] factorial(1) -> 1 
[0.00000000s] factorial(2) -> 2 
[0.00000000s] factorial(3) -> 6 
[0.00000000s] factorial(4) -> 24 
[0.00000000s] factorial(5) -> 120 
[0.00000000s] factorial(6) -> 720 
[0.00000000s] factorial(7) -> 5040 
[0.00000000s] factorial(8) -> 40320 
[0.00000000s] factorial(9) -> 362880 
[0.00000000s] factorial(10) -> 3628800 
[0.00000000s] factorial(11) -> 39916800 
[0.00000000s] factorial(12) -> 479001600 
[0.00000000s] factorial(13) -> 6227020800 
[0.00000000s] factorial(14) -> 87178291200 
[0.00000000s] factorial(15) -> 1307674368000 
[0.00000000s] factorial(16) -> 20922789888000 
[0.00000000s] factorial(17) -> 355687428096000 
[0.00000000s] factorial(18) -> 6402373705728000 
[0.00000000s] factorial(19) -> 121645100408832000 
[0.00000000s] factorial(20) -> 2432902008176640000 
[0.00000000s] factorial(21) -> 51090942171709440000 
[0.00000000s] factorial(22) -> 1124000727777607680000 
[0.00000000s] factorial(23) -> 25852016738884976640000 
[0.00000000s] facto

To run the example,